In [2]:
import requests
import json
import pandas as pd

In [9]:
url = "https://h-amazon-data-scraper2.p.rapidapi.com/products/B08TVXQ5S1/reviews"

querystring = {"api_key":"70201ee0c8ed29661bc6ae00a84341fb"}

headers = {
	"X-RapidAPI-Key": "4da31a08e5mshaca05d98a3d9d6ep1fffb1jsn019717508cc8",
	"X-RapidAPI-Host": "h-amazon-data-scraper2.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"average_rating":4.2,"total_ratings":1518,"filtered_total_ratings":null,"total_reviews":null,"filtered_total_reviews":null,"5_star_ratings":972,"5_star_percentage":0.64,"4_star_ratings":197,"4_star_percentage":0.13,"3_star_ratings":106,"3_star_percentage":0.07,"2_star_ratings":76,"2_star_percentage":0.05,"1_star_ratings":152,"1_star_percentage":0.1,"product":{"url":"https://www.amazon.com/Hydroponics-Growing-QYO-Automation-Starter/dp/B08TVXQ5S1/ref=cm_cr_arp_d_product_top?ie=UTF8","name":"QYO Hydroponics Growing System, 12 Pods Indoor Herb Garden with 36W Full-Spectrum Grow Light, Pump System, Automatic Timer, 23.8'' Height Adjustable, Plants Germination Kit for Home Kitchen Gardening","brand":"QYO","price":null,"image":"https://m.media-amazon.com/images/I/51EWuGFi4EL.jpg","variation":{"Color":"Black"}},"top_positive_review":{"stars":5,"date":"Reviewed in the United States 🇺🇸 on March 23, 2023","verified_purchase":false,"manufacturer_replied":false,"username":"WishGiggles","userUrl":"

In [14]:
response

<Response [200]>

In [19]:
# Assume the JSON data is stored in a variable called `json_data`
data = json.loads(response.text)

In [22]:
# Extract the reviews from the JSON data and convert it to a DataFrame
reviews = pd.DataFrame(data['reviews'])

In [25]:
reviews.to_csv("reviews_sample.csv",index=False)

In [3]:
asin_list_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/external/asin_list.csv'
asin_list = pd.read_csv(asin_list_path)['asin'].tolist()

In [4]:
asin_list

['B08X2324ZL',
 'B09MQ689XL',
 'B0BCW8X98V',
 'B09NR5JW8Y',
 'B0BHS5VPCZ',
 'B0BCVY1DHR',
 'B0BNXCJ3MV',
 'B0BCTTCSBZ',
 'B091325ZMB',
 'B093HCXGC9',
 'B0BL6MR1KN',
 'B09PN11Y65',
 'B0BLNBS36G',
 'B0BVZ3H4MB',
 'B0BW8Y2B8Z',
 'B0BDFZPNVY']

In [7]:
import requests
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Your ASINs
asin_list = [
    'B08X2324ZL', 'B09MQ689XL', 'B0BCW8X98V', 'B09NR5JW8Y', 
    'B0BHS5VPCZ', 'B0BCVY1DHR', 'B0BNXCJ3MV', 'B0BCTTCSBZ', 
    'B091325ZMB', 'B093HCXGC9', 'B0BL6MR1KN', 'B09PN11Y65', 
    'B0BLNBS36G', 'B0BVZ3H4MB', 'B0BW8Y2B8Z', 'B0BDFZPNVY'
]

# Amazon Scraper details
base_url = "https://h-amazon-data-scraper2.p.rapidapi.com/products/"
api_key = "70201ee0c8ed29661bc6ae00a84341fb"
headers = {
    "X-RapidAPI-Key": "4da31a08e5mshaca05d98a3d9d6ep1fffb1jsn019717508cc8",
    "X-RapidAPI-Host": "h-amazon-data-scraper2.p.rapidapi.com"
}

# Firestore details
cred_path = '/Users/vladbordei/Documents/Development/ProductExplorer/notebooks/productexplorerdata-firebase-adminsdk-ulb3d-465f23dff3.json'

# Initialize Firestore
cred = credentials.Certificate(cred_path)
app = firebase_admin.initialize_app(cred)
db = firestore.client()

# Function to get product details
def get_product_details(asin):
    url = f"{base_url}{asin}"
    response = requests.get(url, headers=headers, params={"api_key": api_key})
    return response.json()

# Function to get product reviews
def get_product_reviews(asin):
    url = f"{base_url}{asin}/reviews"
    response = requests.get(url, headers=headers, params={"api_key": api_key})
    return response.json()

# Function to update Firestore
def update_firestore(asin, details, reviews):
    doc_ref = db.collection('products').document(asin)
    doc_ref.set({
        'details': details,
        'reviews': reviews
    })




ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [ ]:
# Loop over ASINs
for asin in asin_list:
    details = get_product_details(asin)
    reviews = get_product_reviews(asin)
    update_firestore(asin, details, reviews)

In [5]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
import aiohttp
import json
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Your ASINs
asin_list = [
    'B08X2324ZL', 'B09MQ689XL', 'B0BCW8X98V', 'B09NR5JW8Y', 
    'B0BHS5VPCZ', 'B0BCVY1DHR', 'B0BNXCJ3MV', 'B0BCTTCSBZ', 
    'B091325ZMB', 'B093HCXGC9', 'B0BL6MR1KN', 'B09PN11Y65', 
    'B0BLNBS36G', 'B0BVZ3H4MB', 'B0BW8Y2B8Z', 'B0BDFZPNVY'
]

# Amazon Scraper details
base_url = "https://h-amazon-data-scraper2.p.rapidapi.com/products/"
api_key = "70201ee0c8ed29661bc6ae00a84341fb"
headers = {
    "X-RapidAPI-Key": "4da31a08e5mshaca05d98a3d9d6ep1fffb1jsn019717508cc8",
    "X-RapidAPI-Host": "h-amazon-data-scraper2.p.rapidapi.com"
}

# Firestore details
cred_path = '/Users/vladbordei/Documents/Development/ProductExplorer/notebooks/productexplorerdata-firebase-adminsdk-ulb3d-465f23dff3.json'

# Initialize Firestore
cred = credentials.Certificate(cred_path)
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)
db = firestore.client()

async def get_product_details(asin):
    async with aiohttp.ClientSession() as session:
        url = f"{base_url}{asin}"
        async with session.get(url, headers=headers, params={"api_key": api_key}) as response:
            return await response.text()

async def get_product_reviews(asin):
    async with aiohttp.ClientSession() as session:
        url = f"{base_url}{asin}/reviews"
        async with session.get(url, headers=headers, params={"api_key": api_key}) as response:
            return await response.text()

def update_firestore(asin, details, reviews):
    doc_ref = db.collection('products').document(asin)
    doc_ref.set({
        'details': json.loads(details),
        'reviews': json.loads(reviews)
    }, merge=True)

async def main():
    for asin in asin_list:
        details = await get_product_details(asin)
        reviews = await get_product_reviews(asin)
        update_firestore(asin, details, reviews)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())


KeyboardInterrupt: 

In [21]:
import time
import asyncio
import nest_asyncio
nest_asyncio.apply()
import aiohttp
import json
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Your ASINs
asin_list = [
    'B08X2324ZL', 'B09MQ689XL', 'B0BCW8X98V'
]

# Amazon Scraper details
# https://rapidapi.com/traject-data-traject-data-default/api/asin-data
# https://rapidapi.com/traject-data-traject-data-default/api/asin-data

base_url = "https://asin-data.p.rapidapi.com/request"
api_key = "70201ee0c8ed29661bc6ae00a84341fb"
headers = {
	"X-RapidAPI-Key": "4da31a08e5mshaca05d98a3d9d6ep1fffb1jsn019717508cc8",
	"X-RapidAPI-Host": "asin-data.p.rapidapi.com"
}

# Firestore details
cred_path = '/Users/vladbordei/Documents/Development/ProductExplorer/notebooks/productexplorerdata-firebase-adminsdk-ulb3d-465f23dff3.json'

# Initialize Firestore
cred = credentials.Certificate(cred_path)
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)
db = firestore.client()

async def get_product_details(asin):
    start = time.time()
    async with aiohttp.ClientSession() as session:
        url = f"{base_url}{asin}"
        async with session.get(url, headers=headers, params={"api_key": api_key}) as response:
            print(f"Fetching product details for {asin} took {time.time() - start} seconds.")
            return await response.text()

async def get_product_reviews(asin):
    start = time.time()
    async with aiohttp.ClientSession() as session:
        params = {
            "type":"reviews",
            "asin":asin,
            "amazon_domain":"amazon.com",
            "reviewer_type":"verified_purchase",
            "sort_by":"most_recent",
            "global_reviews":"false"
        }
        async with session.get(url=base_url, headers=headers, params=params) as response:
            print(f"Fetching product reviews for {asin} took {time.time() - start} seconds.")
            return await response.text()

def update_firestore(asin, details, reviews):
    start = time.time()
    doc_ref = db.collection('products').document(asin)
    doc_ref.set({
        'details': json.loads(details),
        'reviews': json.loads(reviews)
    }, merge=True)
    print(f"Updating Firestore for {asin} took {time.time() - start} seconds.")

async def main():
    for asin in asin_list:
        details = await get_product_details(asin)
        reviews = await get_product_reviews(asin)
        update_firestore(asin, details, reviews)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())


Fetching product details for B08X2324ZL took 0.18781805038452148 seconds.
Fetching product reviews for B08X2324ZL took 0.18324494361877441 seconds.
Updating Firestore for B08X2324ZL took 0.34464406967163086 seconds.
Fetching product details for B09MQ689XL took 0.18163681030273438 seconds.
Fetching product reviews for B09MQ689XL took 0.16463112831115723 seconds.
Updating Firestore for B09MQ689XL took 0.21608924865722656 seconds.
Fetching product details for B0BCW8X98V took 0.15882182121276855 seconds.
Fetching product reviews for B0BCW8X98V took 0.15336203575134277 seconds.
Updating Firestore for B0BCW8X98V took 0.29963207244873047 seconds.


In [22]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Firestore details
cred_path = '/Users/vladbordei/Documents/Development/ProductExplorer/notebooks/productexplorerdata-firebase-adminsdk-ulb3d-465f23dff3.json'

# Initialize Firestore
cred = credentials.Certificate(cred_path)
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred)
db = firestore.client()

# ASIN
asin = 'B09MQ689XL'  # Replace with the ASIN you are interested in

# Get the document with the provided ASIN
doc_ref = db.collection('products').document(asin)

# Get the document data
doc = doc_ref.get()

# Check if document exists
if doc.exists:
    # Get reviews
    reviews = doc.to_dict().get('reviews', {})
    print(reviews)
else:
    print(f"No such document for ASIN: {asin}")


{'2_star_ratings': 71, '3_star_percentage': 0.06, 'average_rating': 4.4, '1_star_ratings': 95, '5_star_percentage': 0.69, '3_star_ratings': 142, 'top_critical_review': {'review': 'I don’t often write bad reviews, but this product is ridiculous. I have been trying to wrap my head around why this thing is as highly reviewed as it is.The first picture is the side I fixed based on another customers review. The second is what the instructions told me to do.It took me two hours to assemble it first, and I quickly realized the instructions left out some really key details. I only discovered those details by watching other customer videos. The instructions are 1/5 stars.I left it alone, thinking I’d just use it as is for the summer. Within two days, the gap in instructions translated to the sides bowing and the gaps in the walls growing.Today I had to spend another hour moving around the dirt I filled it with so I could reconstruct the sides and not destroy what now seems like an overpriced un

In [23]:
len(reviews['reviews'])

10

In [24]:
reviews.keys()

dict_keys(['2_star_ratings', '3_star_percentage', 'average_rating', '1_star_ratings', '5_star_percentage', '3_star_ratings', 'top_critical_review', 'top_positive_review', 'product', 'total_ratings', '5_star_ratings', 'pagination', '4_star_percentage', '2_star_percentage', '4_star_ratings', '1_star_percentage', 'filtered_total_ratings', 'reviews', 'message', 'total_reviews', 'filtered_total_reviews'])

In [28]:
reviews[ 'total_reviews']

583

In [29]:
reviews[ 'filtered_total_reviews']

583